In [1]:
import pandas as pd
import os
import re
from utils import clean_player_name

pd.set_option('display.max_columns', None)

def clean_player_name(player_name):
    if not isinstance(player_name, str):
        return player_name
    player_name = re.sub(r'[^\w\s]', '', player_name)  # Remove punctuation
    suffixes = ['Jr', 'Sr', 'II', 'III', 'IV', 'V']
    pattern = r'\b(?:' + '|'.join(suffixes) + r')\b'
    player_name = re.sub(pattern, '', player_name, flags=re.IGNORECASE)
    return ' '.join(player_name.split())

years = list(range(2016, 2026))
combine_records = []

# Loop over all CSV files
for year in years:
    file_name = f"{year}_Combine.csv"
    if os.path.exists(file_name):
        try:
            df = pd.read_csv(
                file_name,
                encoding='latin1',
                dtype=str,
                parse_dates=False
            )

            if 'Player' not in df.columns:
                print(f"❌ 'Player' column missing in {file_name}, skipping.")
                continue

            df.rename(columns={'Player': 'player'}, inplace=True)
            df['Year'] = str(year)

            if 'Yr' in df.columns:
                df.drop(columns=['Yr'], inplace=True)

            # Fix Drafted column → Split into Team / Round / Pick
            if 'Drafted (tm/rnd/yr)' in df.columns:
                draft_split = df['Drafted (tm/rnd/yr)'].str.extract(
                    r'^(.*?)\s*/\s*(\d+(?:st|nd|rd|th))\s*/\s*(\d+(?:st|nd|rd|th) pick)'
                )
                df['Draft_Team'] = draft_split[0].str.strip()
                df['Draft_Round'] = draft_split[1].str.strip()
                df['Draft_Pick'] = draft_split[2].str.strip()
                df.drop(columns=['Drafted (tm/rnd/yr)'], inplace=True)

            # Filter only QB, RB, WR, TE
            if 'Pos' in df.columns:
                df = df[df['Pos'].isin(['QB', 'RB', 'WR', 'TE'])]

            # Convert Height to inches as float
            if 'Ht' in df.columns:
                def height_to_inches(ht):
                    if not ht or pd.isna(ht):
                        return None
                    ht = str(ht).strip()

                    patterns = [
                        r'^(\d+)[-/](\d+)$',
                        r'^(\d+)\s+(\d+)$',
                        r"^(\d+)'\s*(\d+)$"
                    ]
                    for pattern in patterns:
                        match = re.match(pattern, ht)
                        if match:
                            feet, inches = map(int, match.groups())
                            return feet * 12 + inches

                    month_to_feet = {'may': 5, 'jun': 6}
                    match = re.match(r'^(\d{1,2})[-/](May|Jun)$', ht, flags=re.IGNORECASE)
                    if match:
                        day, month = match.groups()
                        feet = month_to_feet.get(month.lower())
                        inches = int(day)
                        return feet * 12 + inches

                    match = re.match(r'^(May|Jun)[-/](\d{2})$', ht, flags=re.IGNORECASE)
                    if match:
                        month, day = match.groups()
                        feet = month_to_feet.get(month.lower())
                        inches = int(day)
                        return feet * 12 + inches

                    return None

                df['Ht'] = df['Ht'].apply(height_to_inches)

            # Clean player names
            df['player'] = df['player'].apply(clean_player_name)

            combine_records.append(df)
            print(f"✅ Processed {file_name} with {len(df)} rows.")

        except Exception as e:
            print(f"❌ Error reading {file_name}: {e}")
    else:
        print(f"🚫 File not found: {file_name}")

# Combine and format
if combine_records:
    combined_df = pd.concat(combine_records, ignore_index=True)
    combined_df['player'] = combined_df['player'].str.replace('*', '', regex=False)

    # Move 'Year' column directly after 'player'
    cols = list(combined_df.columns)
    if 'Year' in cols:
        cols.remove('Year')
    if 'player' in cols:
        player_index = cols.index('player')
        cols = cols[:player_index + 1] + ['Year'] + cols[player_index + 1:]
        combined_df = combined_df[cols]

    # Create dictionary: player name → DataFrame of their combine records
    player_combine_dict = {
        name: group.reset_index(drop=True)
        for name, group in combined_df.groupby('player')
    }

    print(f"✅ Created player_combine_dict with {len(player_combine_dict)} players.")
else:
    print("❌ No combine data found.")


✅ Processed 2016_Combine.csv with 100 rows.
✅ Processed 2017_Combine.csv with 122 rows.
✅ Processed 2018_Combine.csv with 111 rows.
✅ Processed 2019_Combine.csv with 113 rows.
✅ Processed 2020_Combine.csv with 122 rows.
✅ Processed 2021_Combine.csv with 150 rows.
✅ Processed 2022_Combine.csv with 112 rows.
✅ Processed 2023_Combine.csv with 111 rows.
✅ Processed 2024_Combine.csv with 98 rows.
✅ Processed 2025_Combine.csv with 115 rows.
✅ Created player_combine_dict with 1152 players.


In [3]:
player_combine_dict["Luther Burden"]

,player,Year,Pos,School,College,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Draft_Team,Draft_Round,Draft_Pick
0,Luther Burden,2025,WR,Missouri,College Stats,72.0,206,4.41,NaN,NaN,NaN,NaN,NaN,Chicago Bears,2nd,39th pick
